In [2]:
import torch
print("CUDA disponible:", torch.cuda.is_available())
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


CUDA disponible: True
Nombre de la GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os


False


FileNotFoundError: [Errno 2] No such file or directory: 'LeicaFotos'

In [11]:
pip install polars


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 11.8 MB/s eta 0:00:000:00:010:00:01:02
Note: you may need to restart the kernel to use updated packages.


In [5]:
import polars as pl
import os
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader


In [6]:
img_dir = "LeicaFotos"  # Cambia esta ruta a la ubicación real

image_files = [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f)) 
               and f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif'))]

df_images = pl.DataFrame({
    "image_name": image_files
})

print(df_images)


shape: (1_254, 1)
┌─────────────────────────────────┐
│ image_name                      │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ Leica ICC50 W Camera firmwareV… │
│ Leica ICC50 W Camera firmwareV… │
│ Leica ICC50 W Camera firmwareV… │
│ Leica ICC50 W Camera firmwareV… │
│ Leica ICC50 W Camera firmwareV… │
│ …                               │
│ Leica ICC50 W Camera firmwareV… │
│ Leica ICC50 W Camera firmwareV… │
│ Leica ICC50 W Camera firmwareV… │
│ Leica ICC50 W Camera firmwareV… │
│ Leica ICC50 W Camera firmwareV… │
└─────────────────────────────────┘


In [7]:
class PolarsImageDataset(Dataset):
    def __init__(self, img_dir, df, transform=None):
        self.img_dir = img_dir
        self.df = df
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        img_name = self.df[idx, "image_name"]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, img_name  


In [8]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),  
    transforms.ToTensor(),
])

dataset = PolarsImageDataset(img_dir=img_dir, df=df_images, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)


In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

# cargar imágenes
class UnlabeledImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.image_names = [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f))]
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_names[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# transformes para imágenes
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
])

img_dir = "LeicaFotos"
dataset = UnlabeledImageDataset(img_dir=img_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# arquitectura de la CNn
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 37 * 37, 128)
        self.fc2 = nn.Linear(128, 2)  # Cambia el 2 si tienes más clases en el futuro
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 37 * 37)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = SimpleCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# optimizador y pérdida
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()  # Cambiar según tu objetivo si no tienes etiquetas

# entrenamiento
num_epochs = 10  # Ajustar según tus necesidades
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    running_loss = 0.0
    for images in tqdm(dataloader):
        images = images.to(device)
        optimizer.zero_grad()
        
        outputs = model(images)
        labels = torch.randint(0, 2, (images.size(0),)).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Loss: {running_loss / len(dataloader)}")



Epoch 1/10


100%|███████████████████████████████████████████| 40/40 [01:20<00:00,  2.02s/it]


Loss: 0.7370999783277512
Epoch 2/10


100%|███████████████████████████████████████████| 40/40 [01:17<00:00,  1.95s/it]


Loss: 0.6931892037391663
Epoch 3/10


...

Loss: 0.6935470700263977
Epoch 9/10


100%|███████████████████████████████████████████| 40/40 [01:29<00:00,  2.24s/it]


Loss: 0.6933078080415725
Epoch 10/10


100%|███████████████████████████████████████████| 40/40 [01:29<00:00,  2.23s/it]

Loss: 0.6930192291736603
Entrenamiento completado


In [11]:
torch.save(model.state_dict(), "modelo_tardigrado.pth")
print("Modelo guardado como 'modelo_tardigrado.pth'")



Modelo guardado como 'modelo_tardigrado.pth'
